In [17]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import shutil, os

In [135]:
# Load calib results
proj_mtx =  np.load('../data/proj_mtx.npy', )
proj_dist = np.load('../data/proj_dist.npy', )
cam_mtx =   np.load('../data/cam_mtx.npy', )
cam_dist =  np.load('../data/cam_dist.npy')
proj_R =    np.load('../data/R.npy')
proj_T =    np.load('../data/T.npy')
R1 =        np.load('../data/R1.npy')
R2 =        np.load('../data/R2.npy')
P1 =        np.load('../data/P1.npy')
P2 =        np.load('../data/P2.npy')

# scale_x = 1920 / 800
# scale_y = 1080 / 600
# proj_mtx[0, :] = proj_mtx[0, :] * scale_x
# proj_mtx[1, :] = proj_mtx[1, :] * scale_y
# print(proj_mtx)


base_path = '../data/recordings/record_0/'
h_pixels = np.load(base_path+'h_pixels.npy')
v_pixels = np.load(base_path+'v_pixels.npy')

In [138]:
arrwidth = 1920
arrheight = 1080

cam_pts = []
proj_pts = []

# for i in range(500, 1300):
#     for j in range(200, 800):
for i in range(1920):
    for j in range(1080):
        h_value = h_pixels[j, i]
        v_value = v_pixels[j, i]
        if h_value == -1 or v_value == -1:
            pass
        else:
            cam_pts.append([i,j])
            h_value = min(arrwidth-1, h_value)
            v_value = min(arrheight-1, v_value)
            proj_pts.append([ h_value, v_value])

cam_pts = np.array(cam_pts, dtype=np.float32)
proj_pts = np.array(proj_pts, dtype=np.float32)

newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mtx, cam_dist, (1920,1080), 1, (1920,1080))
newprojmtx, roi = cv2.getOptimalNewCameraMatrix(proj_mtx, proj_dist, (1920,1080), 1, (1920,1080))
cam_pts_homo = cv2.undistortPoints(np.expand_dims(cam_pts, axis=1), cam_mtx, cam_dist, None, newcameramtx)
proj_pts_homo = cv2.undistortPoints(np.expand_dims(proj_pts, axis=1), proj_mtx, proj_dist, None, newprojmtx)

print(proj_pts_homo[0])

[[445.27884 359.43002]]


In [141]:
p4d = cv2.triangulatePoints(P1, P2, cam_pts_homo, proj_pts_homo)

p3d_euc = p4d / p4d[3, :]
p3d_euc = p3d_euc[:3, :]

import open3d as o3d
import open3d.core as o3c
pcd = o3d.t.geometry.PointCloud(o3c.Tensor(p3d_euc.T, o3c.float32))

o3d.visualization.draw_geometries([pcd.to_legacy()])